# Importing libs

In [3]:
from bs4 import BeautifulSoup
import requests
import os
import re
from googletrans import Translator

## parse data

In [4]:
agent = {"User-Agent":"Mozilla/5.0"}
html_data = requests.get('https://www.classcentral.com', headers=agent).text
soup = BeautifulSoup(html_data, 'html.parser')

get the desired imgs

In [102]:
fileToRead = open("relapage.html", "r", encoding="utf8")
soup = BeautifulSoup(fileToRead, 'html.parser')
fileToRead.close()

In [103]:
src_imgs = soup.find_all('img')

In [104]:
len(src_imgs)

54

### getting the wanted to change imgs

In [105]:
with open(".\\ttracker\classcentral\www.classcentral.com\index.html", 'r+',
          encoding="utf8") as file:
    text = file.read()
    txt = re.findall(' src=".*[png gif].*"', text)
print(len(txt))
txt = txt[10:-2]
for i in range(len(txt)):
    txt[i] = txt[i][6:-1]
print(len(txt))
print(txt[0])

66
54
https://ccweb.imgix.net/https%3A%2F%2Fwww.classcentral.com%2Fimages%2Fillustrations%2Flearning-illustration-valentine.png?auto=fयाmat&h=650&ixlib=php-3.3.1&s=fdc8f87b79fd3e2a8a22811688010dc6


### changing the imgs

In [107]:
def change_html_content(replacing_text_list, target_text_list):
    if not len(target_text_list) == len(replacing_text_list):
        print('2 non equal lists')
    with open(".\\ttracker\classcentral\www.classcentral.com\index.html", 'r+',
          encoding="utf8") as file:
        text = file.read()
        for i in range(len(src_imgs)):
            text = text.replace(target_text_list[i], replacing_text_list[i])
        file.seek(0)
        file.write(text)
        file.truncate()

In [108]:
with open(".\\ttracker\classcentral\www.classcentral.com\index.html", 'r+',
          encoding="utf8") as file:
    text = file.read()
    for i in range(len(src_imgs)):
        text = text.replace(txt[i], src_imgs[i]['src'])
    file.seek(0)
    file.write(text)
    file.truncate()
    

In [ ]:
with open(os.path.join('test.html'), 'w',
              encoding="utf-8") as dictfile:
        dictfile.write(soup)

## translating the pages to hindi
- we need to get the wanted text
- text is inside tages or the contents of tags
- we need to translate it
- we need to replace it

### get the required tags

In [114]:
def get_tags(soup):
  tags = ['p', 'a', 'h1', 'h2', 'h3', 'h4', 'title', 'button']
  # for x in soup.find_all():
  #   tags.append(x.name)
  tags = set(tags)
  # tags_list = ['style', 'html', 'body', 'script', 'head', 'meta',
  #              'header', 'footer', 'img', 'section', 'link']
  # for tag in tags_list:
  #   if tag in tags:
  #     tags.remove(tag)
  return tags

### get the sentences which we want to translate

In [161]:
def get_sentences(tags, soup):
    sentences = []
    if soup.find('title').text == 'Just a moment...':
        return ''
    print(soup.find('title').text)
    for tag in tags:
        tag_list = soup.find_all(tag)
        for i in tag_list:
            if i.text:
                sentences.append(re.split('\n+', i.text))
    return sentences

### translate every sentences

In [8]:
translator = Translator(service_urls=['translate.googleapis.com'])

result = translator.translate('हार्वर्ड', src='hi', dest='en')

print(result.src)
print(result.dest)
print(result.text)

hi
en
Harvard


In [177]:
x = [1,2,3]
def translate(sentences, file_path):
    print('translating')
    translator = Translator(service_urls=['translate.googleapis.com'])
    if 'or' in sentences:
        sentence = sentences.remove('or')
    if 'harvard' in sentences:
        sentence = sentences.remove('harvard')
    with open(".\\ttracker\classcentral\www.classcentral.com\\"+file_path, 'r+',
            encoding="utf8") as file:
        text = file.read()
        for sentence in sentences:
            if type(sentence) == type(x):
                if x in sentence:
                    sentence = sentence.remove(x)
                for s in sentence: 
                    if s == '&nbsp;':
                        continue                       
                    if s is None:
                        continue
                    if s == '':
                        continue
                    if s.isspace():
                        continue
                    else:
                        print(s)
                        result = translator.translate(str(s), src='en', dest='hi')
                        text = text.replace(s, result.text)
                        
            else:
                if s == '&nbsp;':
                        continue  
                if len(sentence) == 0:
                        continue
                if sentence.isspace():
                        continue
                if sentence is None:
                        continue
                
                else:
                    print(sentence)
                    result = translator.translate(sentence, src='en', dest='hi')
                    text = text.replace(sentence, result.text)

        file.seek(0)
        file.write(text)
        file.truncate()

### Full translation function


In [31]:
def get_soup(file_path):
    fileToRead = open(".\\ttracker\classcentral\www.classcentral.com\\"+file_path, "r", encoding="utf8")
    soup = BeautifulSoup(fileToRead, 'html.parser')
    fileToRead.close()
    return soup

In [209]:
def download_soup(file_path):
    with open(".\\ttracker\classcentral\www.classcentral.com\\"+file_path, 'r+',
            encoding="utf8") as file:
        text = str(get_soup(file_path))
        file.seek(0)
        file.write(text)
        file.truncate()

In [57]:
def full_translate(file_path):
    print('working on ' + file_path)
    soup = get_soup(file_path)
    tags = get_tags(soup)
    sentences = get_sentences(tags, soup)
    translate(sentences, file_path)

In [14]:
ori_soup = get_soup('\index.html')

In [182]:
a_tags = ori_soup.find_all('a')
i = 1
for tag in a_tags:
    if 'index.html' == tag['href']:
        continue
    else:
        print(i)
        full_translate(tag['href'])
        i += 1